# **Init**

In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
from bbmodcache.bbSearch import SearchProblem, search

class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength
        
    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])
    
    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location, 
                      self.carried_items,
                      self.strength ) )
            
class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}
        
    ## Define a unique string representation for a door object    
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

class State:
    def __init__( self, robot, doors, room_contents, room_locations ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents
        self.room_locations = room_locations
        
    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of 
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )

ROOM_CONTENTS = {
    'component room': {'bolt', 'gear', 'spring', 'white key'},
    'store room': {'bucket', 'suitcase', 'toolbox'},
    'tool cupboard': {'sledge hammer', 'anvil', 'saw', 'screwdriver'},
    'power tool room': {'drill', 'cutter', 'polisher'},
    'workshop': {'helmet', 'glasses'},
    'break room': {'water', 'snacks', 'tissues', 'black key'}
}

ROOM_LOCATIONS = {
    'component room': (1, 1),
    'store room': (2, 1),
    'tool cupboard': (2, 2),
    'power tool room': (3, 2),
    'workshop': (3, 1),
    'break room': (2, 3)
}

ITEM_WEIGHT = {
      'white key': 0,
      'black key': 0,
      'bolt': 10,
      'gear': 2,
      'spring': 10,
      'bucket': 4,
      'suitcase': 6,
      'toolbox': 4,
      'sledge hammer': 5,
      'anvil': 12,
      'saw': 2,
      'screwdriver': 1,
      'drill': 6,
      'cutter': 10,
      'polisher': 8,
      'helmet': 3,
      'glasses':2,
      'water': 1,
      'snacks': 3,
      'tissues': 1         
}
          
DOORS = [
    Door('store room', 'workshop', doorkey='black key', locked=True),
    Door('power tool room', 'workshop', doorkey='white key', locked=True),
    Door('component room', 'store room' ),
    Door('store room', 'tool cupboard'),
    Door('tool cupboard', 'power tool room'),
    Door('tool cupboard', 'break room')
]    

from copy import deepcopy

class RobotWorker( SearchProblem ):
    
    def __init__( self, state, goal_item_locations):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations
        self.lock_counter = 0
        
    def possible_actions( self, state ):
        
        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()
        
        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough    
        for i in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHT[i]:
                actions.append( ("pick up", i))
                
        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
            # If there is an locked door between robot location and
            # another location and the robot carries key
            if robot_location in door.goes_between and door.locked==True:
              for i in state.robot.carried_items:
                if i == door.doorkey:
                  actions.append(("open to", door.other_loc[robot_location]))
                  actions.append( ("move to", door.other_loc[robot_location]) )
          
        # Now the actions list should contain all possible actions
        return actions
    
    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)
            
        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)
            
        if act == "move to":
            next_state.robot.location = target
        
        if act == 'open to':
          self.lock_counter += 1
            
        return next_state
        
    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True
    
    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)
    
    def manhattan(self, state):
      # find all goal items
      items = []
      for room in self.goal_item_locations:
        # find the goal locationn
        goal_location = room
        for item in self.goal_item_locations[room]:
          items.append(item)

      
      initial_locations = []
      # find the current location of item
      for goal_item in items:
        find = False
        for room in state.room_contents:
          for item in state.room_contents[room]:
            if item == goal_item:
              find = True
              break
          if find:
            break
        if find:
          initial_locations.append(room)
        else:
          initial_locations.append('robot')

      # print('initial locations', initial_locations)
      # for room in state.room_contents:
      #   for initial_item in state.room_contents[room]:
      #     for goal_item in items:
      #       if initial_item == goal_item:
      #         initial_locations.append(room)

      # calculate the coordinates of the robot location
      robotx, roboty = state.room_locations[state.robot.location]
      # calculate the coordinates of the goal location
      goalx, goaly = state.room_locations[goal_location]
      manhattan = []

      # calculate the coordinates of the initial location
      for initial_location in initial_locations:
        distance = 0
        if initial_location is 'robot':
          distance += abs(robotx - goalx)
          distance += abs(roboty - goaly)
        elif initial_location is goal_location:
          distance = 0
        else:
          initx, inity = state.room_locations[initial_location]
          # calculate the manhattan distance between the 
          # initial location and the robot
          distance += abs(initx - robotx)
          distance += abs(inity - roboty)

          # calculate the manhattan distance between the initial 
          # location and the goal location
          distance += abs(initx - goalx)
          distance += abs(inity - goaly)
        manhattan.append(distance)

      maxManhattan = max(manhattan) + 1
      return maxManhattan


rob = Robot('break room', [], 10)
state = State(rob, DOORS, ROOM_CONTENTS, ROOM_LOCATIONS)
goal_item_locations = {"power tool room": {"water", "snacks"}}
SIMPLE_PRO = RobotWorker(state, goal_item_locations)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0  50997      0 --:--:-- --:--:-- --:--:-- 50859
Loading bbSearch Version 2.1 (at 18:20, Mon 14 Mar)
Last module source code edit 9am Thursday 24th Feb 2022


# **Simple Result**

In [ ]:
# FAST
T1_1 = search(SIMPLE_PRO, 'BF/FIFO', 50000, loop_check=True, return_info=True)
T1_2 = search(SIMPLE_PRO, 'DF/LIFO', 100000, loop_check=True, return_info=True)
T1_3 = search(SIMPLE_PRO, 'DF/LIFO', 100000, loop_check=True, randomise=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...
:-)) *SUCCESS* ((-:

Path length = 6
Goal state is:
Robot location: power tool room
Robot carrying: []
Room contents: {'component room': {'bolt', 'white key', 'gear', 'spring'}, 'store room': {'bucket', 'toolbox', 'suitcase'}, 'tool cupboard': {'sledge hammer', 'anvil', 'screwdriver', 'saw'}, 'power tool room': {'polisher', 'snacks', 'water', 'drill', 'cutter'}, 'workshop': {'helmet', 'glasses'}, 'break room': {'tissues', 'black key'}}
The action path to the solution is:
    ('pick up', 'snacks')
    ('pick up', 'water')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('put down', 'snacks')
    ('put down', 'water')


SEARCH SPACE STATS:
Total nodes generat

In [ ]:
def cost(path, state):
  return len(path)

In [ ]:
# FAST
T1_4 = search(SIMPLE_PRO, 'BF/FIFO', 100, heuristic=SIMPLE_PRO.manhattan, loop_check=True, return_info=True)
T1_5 = search(SIMPLE_PRO, 'BF/FIFO', 2000, cost=cost, heuristic=SIMPLE_PRO.manhattan, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=manhattan
Max search nodes: 100  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 6
Goal state is:
Robot location: power tool room
Robot carrying: []
Room contents: {'component room': {'bolt', 'white key', 'gear', 'spring'}, 'store room': {'bucket', 'toolbox', 'suitcase'}, 'tool cupboard': {'sledge hammer', 'anvil', 'screwdriver', 'saw'}, 'power tool room': {'polisher', 'snacks', 'water', 'drill', 'cutter'}, 'workshop': {'helmet', 'glasses'}, 'break room': {'tissues', 'black key'}}
The action path to the solution is:
    ('pick up', 'snacks')
    ('pick up', 'water')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('put down', 'snacks')
    ('put down', 'water')


SEARCH SPACE STATS:
Total nodes generat

In [ ]:
AL_NAME = ['BFS_LT', 'DFS_LT_RF', 'DFS_LT_RT', 'BEST_1ST', 'A*']
TEST_RESULTS ={'BFS_LT': T1_1, 
               'DFS_LT_RF': T1_2, 
               'DFS_LT_RT': T1_3,  
               'BEST_1ST': T1_4, 
               'A*': T1_5}

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    # cost = len(path)
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:      50000    Y     22124     9988       7.45    6
 DFS_LT_RF:     100000    Y      7710     5362       3.05    877
 DFS_LT_RT:     100000    Y      8749     5903        4.0    1413
  BEST_1ST:        100    Y        71       43       0.04    6
        A*:       2000    Y      1181      500       0.44    6


# **Moderate Result**

In [ ]:
rob = Robot('break room', [], 10)
state = State(rob, DOORS, ROOM_CONTENTS, ROOM_LOCATIONS)
goal_item_locations = {"workshop": {"gear"}}
MODERATE_PRO = RobotWorker(state, goal_item_locations)

In [ ]:
def cost(path, state):
  return len(path)

In [ ]:
# a little long
T2_1 = search(MODERATE_PRO, 'BF/FIFO', 800000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 800000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...................................................................
:-)) *SUCCESS* ((-:

Path length = 8
Goal state is:
Robot location: workshop
Robot carrying: ['black key']
Room contents: {'component room': {'bolt', 'spring', 'white key'}, 'store room': {'bucket', 'suitcase', 'toolbox'}, 'tool cupboard': {'screwdriver', 'saw', 'anvil', 'sledge hammer'}, 'power tool room': {'polisher', 'cutter', 'drill'}, 'workshop': {'gear', 'glasses', 'helmet'}, 'break room': {'water', 'snacks', 'tissues'}}
The action path to the solution is:
    ('pick up', 'black key')
    ('move to', 'tool cupboard')
    ('move to', 'store room')
    ('move to', 'component room')
    ('pick up', 'gear')
    ('m

In [ ]:
T2_2 = search(MODERATE_PRO, 'DF/LIFO', 200000, loop_check=True, return_info=True)

流式输出内容被截断，只能显示最后 5000 行内容。
    ('move to', 'tool cupboard')
    ('move to', 'store room')
    ('move to', 'component room')
    ('put down', 'glasses')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('pick up', 'snacks')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('pick up', 'helmet')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('put down', 'helmet')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('pick up', 'water')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('pick up', 'helmet')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('put down', 'snacks')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('put down', 'water')
    ('move to', 'tool cupboard')
    ('move to', 'store room')
    ('move to', 'component room')
    ('pick up', 'glasses')
    ('move to', 'store room')
    ('move to', 'tool cup

In [ ]:
# short
T2_3 = search(MODERATE_PRO, 'DF/LIFO', 5000, loop_check=True, randomise=True, return_info=True)
T2_4 = search(MODERATE_PRO, 'BF/FIFO', 5000, heuristic=MODERATE_PRO.manhattan, loop_check=True, return_info=True)
T2_5 = search(MODERATE_PRO, 'BF/FIFO', 50000, cost=cost, heuristic=MODERATE_PRO.manhattan, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 616
Goal state is:
Robot location: workshop
Robot carrying: ['tissues', 'black key']
Room contents: {'component room': {'bolt'}, 'store room': {'water', 'suitcase', 'white key', 'sledge hammer', 'polisher', 'snacks'}, 'tool cupboard': {'anvil', 'spring'}, 'power tool room': {'toolbox', 'glasses', 'bucket', 'screwdriver', 'cutter', 'helmet', 'saw'}, 'workshop': {'gear', 'drill'}, 'break room': set()}
The action path to the solution is:
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('pick up', 'polisher')
    ('move to', 'tool cupboard')
    ('pick up', 'screwdriver')
    ('move to', 'break room')
    ('pick up', 'black key')


In [ ]:
AL_NAME = ['BFS_LT', 'DFS_LT_RF', 'DFS_LT_RT', 'DFS_LF_RF', 'BEST_1ST', 'A*']
TEST_RESULTS ={'BFS_LT': T2_1, 
               'DFS_LT_RF': T2_2, 
               'DFS_LT_RT': T2_3,  
               'BEST_1ST': T2_4, 
               'A*': T2_5}

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    # cost = len(path)
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:     800000    Y    421043   147774     139.68    8
 DFS_LT_RF:     200000    Y     47093    33316      35.36    6176
 DFS_LT_RT:       5000    Y      3850     2546       1.33    616
  BEST_1ST:       5000    Y      1396      395       0.53    10
        A*:      50000    Y      5481     2899       7.58    8


# **Difficult Problem**

In [ ]:
rob = Robot('power tool room', [], 10)
state = State(rob, DOORS, ROOM_CONTENTS, ROOM_LOCATIONS)
goal_item_locations = {"workshop": {'drill', 'screwdriver'}}
DIFFICULT_PRO = RobotWorker(state, goal_item_locations)

In [ ]:
T3_1 = search(DIFFICULT_PRO, 'BF/FIFO', 400000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 400000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
...........................
!! Search node limit (400000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   763470  (includes start)
Nodes discarded by loop_check  =   363469  (400001 distinct states added to queue)
Nodes tested (by goal_test)    =   127968  (all expanded)
Nodes left in queue            =   272032

Time taken = 264.74 seconds



In [ ]:
def cost(path, state):
  return len(path)

In [ ]:
T3_2 = search(DIFFICULT_PRO, 'DF/LIFO', 100000, loop_check=True, return_info=True)
T3_3 = search(DIFFICULT_PRO, 'DF/LIFO', 100000, loop_check=True, randomise=True, return_info=True)

流式输出内容被截断，只能显示最后 5000 行内容。
    ('open to', 'store room')
    ('move to', 'power tool room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('pick up', 'saw')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('move to', 'tool cupboard')
    ('move to', 'power tool room')
    ('move to', 'workshop')
    ('open to', 'store room')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('put down', 'saw')
    ('move to', 'break room')
    ('put down', 'screwdriver')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    ('move to', 'break room')
    ('move to', 'tool cupboard')
    (

In [ ]:
T3_4 = search(DIFFICULT_PRO, 'BF/FIFO', 20000, heuristic=DIFFICULT_PRO.manhattan, loop_check=True, return_info=True)
T3_5 = search(DIFFICULT_PRO, 'BF/FIFO', 40000, cost=cost, heuristic=DIFFICULT_PRO.manhattan, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=manhattan
Max search nodes: 20000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)


TypeError: ignored

# **Summary**

In [ ]:
AL_NAME = ['BFS_LT', 'DFS_LT_RF', 'DFS_LT_RT', 'DFS_LF_RF', 'BEST_1ST', 'A*']
TEST_RESULTS ={'BFS_LT': T3_1, 
        'DFS_LT_RF': T3_2, 
        'DFS_LT_RT': T3_3,  
        'BEST_1ST': T3_4, 
        'A*': T3_5}

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    # cost = len(path)
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:     400000    !    763470   272032     264.74    !
 DFS_LT_RF:     100000    Y     36109    24471      25.19    4515
 DFS_LT_RT:     100000    Y     16301    10836      11.24    2788
  BEST_1ST:      20000    !     44372    11639       18.4    !
        A*:      40000    !     71994    27718      26.01    !
